# Commodity List
- **Reference**: https://site.financialmodelingprep.com/developer/docs#full-quote-list-commodities

## Step 1: Import Libraries

In [1]:
%run ../make_clean_names.py

In [2]:
import os
import logging
import requests
import polars as pl
from datetime import datetime
from typing import Dict
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def create_session() -> requests.Session:
    """Create a session with retry strategy."""
    session = requests.Session()
    retries = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504]
    )
    session.mount('https://', HTTPAdapter(max_retries=retries, pool_maxsize=10))
    return session

# Get API key from environment variables
FMP_API_KEY = os.getenv('FMP_API_KEY')
if not FMP_API_KEY:
    raise ValueError("FMP_API_KEY not found in environment variables")

## Step 2: Extract Data from FMP into Polars

In [3]:
def fetch_data(api_key: str, session: requests.Session) -> Dict:
    """Fetch historical forex data for a single pair with pair identifier"""
    url = f"https://financialmodelingprep.com/api/v3/quotes/commodity"
    params = {
        "apikey": api_key
    }
        
    try:
        response = session.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        return data
    except Exception as e:
        return None

# Create a session
session = create_session()

# Execute fetching
data = fetch_data(FMP_API_KEY, session)

# Convert to Polars DataFrame if data exists
if data:
    df = pl.DataFrame(data)

## Step 3: Clean Column Names

In [4]:
df = make_clean_names(df)

## Step 4: Write Polars to Parquet

In [5]:
# Define the output directory
output_dir = "../../../data/finance"

# Write the processed DataFrame to a Parquet file
df.write_parquet(f'{output_dir}/commodity_list.parquet')

## Step 5: Read Parquet (Validate)

In [6]:
# Validate the output by reading the Parquet file and displaying the first few rows
pl.scan_parquet(f'{output_dir}/commodity_list.parquet').head().collect()

symbol,name,price,changes_percentage,change,day_low,day_high,year_high,year_low,market_cap,price_avg_50,price_avg_200,exchange,volume,avg_volume,open,previous_close,eps,pe,earnings_announcement,shares_outstanding,timestamp
str,str,f64,f64,f64,f64,f64,f64,f64,null,f64,f64,str,i64,i64,f64,f64,null,null,null,null,i64
"""DCUSD""","""Class III Milk Futures""",20.27,-0.14778,-0.03,19.27,20.28,23.43,15.17,null,null,null,"""COMMODITY""",274,232,20.28,20.3,null,null,null,null,1737127140
"""RTYUSD""","""Micro E-mini Russell 2000 Inde…",2292.7,0.59231,13.5,2273.9,2307.5,2477.1,1915.8,null,2336.677,2182.1152,"""COMMODITY""",12660,159874,2275.3,2279.2,null,null,null,null,1737127238
"""ZLUSX""","""Soybean Oil Futures""",45.14,0.24428,0.11,44.95,45.69,49.8,38.6,null,42.6814,43.55505,"""COMMODITY""",8599,46734,45.24,45.03,null,null,null,null,1737127236
"""LEUSX""","""Live Cattle Futures""",196.4,-0.10173,-0.2,195.725,198.525,199.1,172.7,null,189.8195,185.00862,"""COMMODITY""",26049,15706,196.925,196.6,null,null,null,null,1737127235
"""DXUSD""","""US Dollar""",108.8,-0.015623,-0.017,108.68,109.255,110.015,99.865,null,107.04008,104.4854,"""COMMODITY""",8240,19682,108.785,108.817,null,null,null,null,1737127235
